# CEPEA - Web Scraping - Diárias do Mercado - Açúcar e Etanol

Esse _script_ visa obter todos os URL da seção "Diárias do Mercado" do website do CEPEA que possua algum boletim sobre Açúcar ou Etanol.

* **Observação:** Esse ele deve ser executado apenas uma única vez para criar o banco de dados inicial.

## 1. Estratégia

Todas as "Diárias do Mercado" que iniciarem com as expressões: `AÇÚCAR/CEPEA` ou `ETANOL/CEPEA` serão armazenadas.

## 2. Requisitos

Parte do site usa `JavaScript`, por este motivo, não será possível obter as informações somente usando a biblioteca [Beautiful Soup](https://pypi.org/project/beautifulsoup4/) (`bsf4`). Foi necessário a utilização do [Selenium](https://pypi.org/project/selenium/) para iterar entre as páginas do _feed_.

### 2.1. Bibliotecas

Conforme já supracitado iremos usar o Selenium.

* `Selenium`: Acesso às informações via web browser do Chrome;
* `pandas`: Maninupalação de dataframe, e;
* `time`: Manipulação de datas.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import os
import time

## 3. Inicialização das Variáveis

In [ ]:
# Base URL com a estrutura do site
base_url = "https://www.cepea.esalq.usp.br/br/diarias-de-mercado/acucar-cepea-com-baixa-liquidez-precos-tem-leve-queda.aspx?pagina={}"

# Listas para armazenar os dados
dates = []
products = []
titles = []
links = []

## 4. Função

Checagem das publicações pertencem a `AÇÚCAR/CEPEA` ou `ETANOL/CEPEA`.

In [ ]:
# Função para verificar se um título pertence aos produtos de interesse
def is_product_of_interest(title):
    return title.startswith("AÇÚCAR/CEPEA") or title.startswith("ETANOL/CEPEA")

## 5. Web Driver

Abre o navegador `Chrome` e inicia a navegação para aquisição dos URLs.

In [ ]:
# Configuração do WebDriver (certifique-se de que o chromedriver está no PATH ou forneça o caminho completo)
driver = webdriver.Chrome()

# Iterar pelas páginas de 1 a 84
for page in range(1, 85):
    # Construir a URL da página atual
    url = base_url.format(page)
    
    # Navegar para a página
    driver.get(url)
    
    # Esperar alguns segundos para garantir que o JS carregue o conteúdo
    time.sleep(5)  # Ajuste o tempo de acordo com a velocidade de carregamento da página
    
    # Encontrar o div com os links
    try:
        div = driver.find_element(By.CLASS_NAME, 'imagenet-listagem-texto')
        
        # Encontrar todos os links dentro do div
        link_tags = div.find_elements(By.CLASS_NAME, 'box-texto-relacionado')
        
        for link_tag in link_tags:
            link = link_tag.get_attribute('href')
            title_span = link_tag.find_element(By.TAG_NAME, 'span').text.strip()
            title = link_tag.text.strip().split(' - ')[1]  # Extrai o título
            
            # Verificar se o título pertence a AÇÚCAR/CEPEA ou ETANOL/CEPEA
            if is_product_of_interest(title):
                # Determinar o produto baseado no título
                if title.startswith("AÇÚCAR/CEPEA"):
                    product = "AÇÚCAR/CEPEA"
                    title = title.replace("AÇÚCAR/CEPEA: ", "")
                elif title.startswith("ETANOL/CEPEA"):
                    product = "ETANOL/CEPEA"
                    title = title.replace("ETANOL/CEPEA: ", "")
                
                # Extrair a data
                pub_date = link_tag.find_element(By.CLASS_NAME, 'data').text.strip()
                
                # Adicionar os dados às listas
                dates.append(pub_date)
                products.append(product)
                titles.append(title)
                links.append(link)
    except Exception as e:
        print(f"Erro ao processar a página {page}: {e}")

# Fechar o navegador
driver.quit()

## 6. DataFrame

Cria o dataframe `df` que será usado para exportação.

In [ ]:
# Criar o DataFrame com os dados coletados
df = pd.DataFrame({
    'Data': dates,
    'Produto': products,
    'Título': titles,
    'URL': links
})

df.head()

## 3. Exportação

Cria o arquivo inicial com URL, data e título.

In [ ]:
# Caminho completo para o arquivo de exportação
file_path = os.path.join("01-data", time.strftime('%Y-%m-%d') + ' ' + 'cepea_feed_list.csv')

# Exporta na pasta 01-data
df.to_csv(file_path, index=False, encoding='utf-8-sig')